In [ ]:
#### IMPORTS ####
using ProgressMeter
using Plots, Measures
using LaTeXStrings
using NPZ
theme(:default)
include("dWaveRgFlow.jl")

In [ ]:
map1DTo2D(points, num_kspace) = [(points .- 1) .% num_kspace .+ 1, div.((points .- 1), num_kspace) + 1]

num_kspace_half = 40
num_kspace = 2 * num_kspace_half + 1
kx_arr = range(-pi, stop=pi, length=num_kspace)
FermiArmSouthWest = [num_kspace_half + 1 + j * (num_kspace - 1) for j in 0:(num_kspace_half)]
FermiArmAbove = [point - 1 for point in FermiArmSouthWest if 1 <= (point - 1) % num_kspace <= num_kspace_half]
FermiArmBelow = [point + 1 for point in FermiArmSouthWest if 1 <= point % num_kspace <= num_kspace_half]

antinodeD = FermiArmSouthWest[1]
midway = FermiArmSouthWest[div(1 + num_kspace_half, 6) + 1]
node = FermiArmSouthWest[div(1 + num_kspace_half, 2) + 1]

node_kx, node_ky = round.(kx_arr[map1DTo2D(node, num_kspace)] ./ pi, digits=2)
midway_kx, midway_ky = round.(kx_arr[map1DTo2D(midway, num_kspace)] ./ pi, digits=2)
antinodeD_kx, antinodeD_ky = round.(kx_arr[map1DTo2D(antinodeD, num_kspace)] ./ pi, digits=2)

# println(antinodeD, " ", midway, " ", node)
# kx_arr = range(-pi, stop=pi, length=num_kspace)
J_val = 0.1

# kondoJArray, dispersionArray = main(num_kspace_half, 1, J_val, -0)
# println(kondoJArray[21,:,1])

# W_val_range = [0.004, 0.008, 0.01, 0.02, 0.05, 0.055, 0.06]
W_val_range = 0:1:1
plots1 = repeat([plot(1)], length(W_val_range) * 3)
# plots2 = repeat([plot(1)], length(W_val_range) * 3)
for (j, W_val) in collect(enumerate(J_val .* W_val_range))
    @time kondoJArray, dispersionArray = main(num_kspace_half, 1, J_val, -W_val)
    J_antinodeD = zeros(length(FermiArmAbove), num_kspace_half+1)
    J_midway = zeros(length(FermiArmAbove), num_kspace_half+1)
    J_node = zeros(length(FermiArmAbove), num_kspace_half+1)
    for i in 1:(num_kspace_half+1)
        J_antinodeD[:,i] = kondoJArray[antinodeD,FermiArmAbove,i]
        J_midway[:,i] = kondoJArray[midway,FermiArmAbove,i]
        J_node[:,i] = round.(kondoJArray[node,FermiArmAbove,i], digits=10)
    end

    title_node = "\$\\vec k_1 = ($node_kx\\pi, $node_ky\\pi), \\quad W/t=$W_val, \\quad J/t=$J_val\$"
    title_antinode = "\$\\vec k_1 = ($antinodeD_kx\\pi, $antinodeD_ky\\pi), \\quad W/t=$W_val, \\quad J/t=$J_val\$"
    title_midway = "\$\\vec k_1 = ($midway_kx\\pi, $midway_ky\\pi), \\quad W/t=$W_val, \\quad J/t=$J_val\$"

    plots1[(j-1) * 3 + 1] = heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), J_antinodeD, title=title_antinode, c=:thermal)
    plots1[(j-1) * 3 + 2] = heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), J_midway, title=title_midway, c=:thermal)
    plots1[(j-1) * 3 + 3] = heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), J_node, title=title_node, c=:thermal)
    
    # push!(plots2, heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), log10.(abs.(J_node)), title="\$\\vec k_1 = (0,-\\pi), \\quad W=$W_val, \\quad J/t=$J_val\$", c=:thermal))
    # push!(plots2, heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), log10.(abs.(J_midway)), title="\$\\vec k_1 = (-\\pi/4,-3\\pi/4), \\quad W=$W_val, \\quad J/t=$J_val\$", c=:thermal))
    # push!(plots2, heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), log10.(abs.(J_antinodeD)), title="\$\\vec k_1 = (-\\pi/2,-\\pi/2), \\quad W=$W_val, \\quad J/t=$J_val\$", c=:thermal))
    
end

p= plot(plots1..., layout=(length(W_val_range), 3), size=(400*3, 200*length(W_val_range)), titlefontsize=10, topmargin=0mm, bottommargin=0mm, 
    leftmargin=3mm, rightmargin=3mm)
savefig(p, "figCombined.pdf")
# p= plot(plots2..., layout=(length(W_val_range), 3), size=(400*3, 200*length(W_val_range)), titlefontsize=10, topmargin=0mm, bottommargin=0mm, 
#     leftmargin=3mm, rightmargin=3mm)
# savefig(p, "figCombined_abs.pdf")

In [ ]:
num_kspace_half = 50
num_kspace = 2 * num_kspace_half + 1
kx_arr = range(-pi, stop=pi, length=num_kspace)
FermiArmSouthWest = [num_kspace_half + 1 + j * (num_kspace - 1) for j in 0:(num_kspace_half)]
FermiArmAbove = [point - 1 for point in FermiArmSouthWest if 1 <= (point - 1) % num_kspace <= num_kspace_half]

J_val = 0.02

W_val_range = 0:0.3:1
antinodeD = FermiArmSouthWest[end]
midway = FermiArmSouthWest[div(1 + num_kspace_half, 4) + 1]
node = FermiArmSouthWest[div(1 + num_kspace_half, 2)]
fspoints = [antinodeD, midway, node]
points = [FermiArmAbove[FermiArmSouthWest[2:end] .== point][1] for point in fspoints]
println(points)

for W_val in J_val .* W_val_range
    plot1 = plot(thickness_scaling=1.5, size=(800, 300), layout=(1, 2), palette = :tol_bright, xlabel="RG flow", ylabel="Kondo coupling", title="\$W=$W_val\$")
    @time kondoJArray, dispersionArray = main(num_kspace_half, 1, J_val, -W_val)
    for (p1, p2, label) in zip(fspoints, points, ["AN", "M", "N"])
        plot!(plot1, kondoJArray[p1, p2, :], label=label, linewidth = 4, subplot=1)
        plot!(plot1, kondoJArray[p1, p1, :], label=label, linewidth = 4, subplot=2)
    end
    display(plot1)
    # display(plot2)
end

# savefig(plot1, "figCombinedForward.pdf")
# p= plot(plots2..., layout=(length(W_val_range), 3), size=(400*3, 200*length(W_val_range)), titlefontsize=10, topmargin=0mm, bottommargin=0mm, 
#     leftmargin=3mm, rightmargin=3mm)
# savefig(p, "figCombined_abs.pdf")